In [1]:
import os
import glob
import zlib
import pathlib
import tempfile
import subprocess
import itertools
import random
import math
import scipy as sp
import scipy.optimize
import cma
import skimage.draw
from scipy.optimize import differential_evolution, minimize, basinhopping
from sklearn.metrics import pairwise_distances
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


In [2]:
input_glob = os.path.join("..", "resources", "islands", "*.npy")

In [3]:
input_file = os.path.join("..", "resources", "islands", "moderate_l_01.npy")
input_data = np.load(input_file)

In [4]:
# flip axes
input_data = input_data.T

In [5]:
px.imshow(input_data)

In [6]:
num_sources = 24
radius = 20
source_size = 4
e = 1e-8

In [7]:
img = np.zeros((40, 40))
img[skimage.draw.disk(
    center=(19.5, 19.5),
    radius=20
)] = 1
px.imshow(img)

In [8]:
townhall_coverage = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0 ],
    [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0 ],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0 ],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0 ],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0 ],
    [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0 ],
    [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
])
townhall_disk = np.nonzero(townhall_coverage)
px.imshow(townhall_coverage)

In [9]:
num_variables = input_data.ndim * num_sources
bounds_lower = [0 for _ in range(num_variables)]
bounds_upper = [input_data.shape[d] - 1 for _ in range(num_sources) for d in range(input_data.ndim)]
bounds = list(zip(bounds_lower, bounds_upper))
bounds

[(0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383),
 (0, 383)]

In [10]:
# remove surrounding ocean
interior = np.nonzero(input_data)
input_data = input_data[
    interior[0].min():interior[0].max()+1,
    interior[1].min():interior[1].max()+1,
]

In [11]:
def distance_from_set(s) -> np.ndarray:
    distance = np.where(s, 0.0, np.inf)
    visited = np.asarray(s).astype(bool).copy()
    for k in itertools.count():
        visited = e < scipy.ndimage.uniform_filter(visited.astype(float), 3)
        mask = (distance == np.inf) & visited
        if not mask.any():
            break
        distance[mask] = 1 + k
    return distance

def signed_distance_from_set(s) -> np.ndarray:
    return distance_from_set(s) - np.maximum(0, -1 + distance_from_set(~s))

In [12]:
input_center = (input_data.shape[0] / 2, input_data.shape[1] / 2)
distance_grid = signed_distance_from_set(input_data)
distance_max = distance_grid.max()
px.imshow(distance_grid)

In [13]:
def vector_to_sources(x):
    return x.reshape((-1, 2))

def sources_to_grid(sources):
    a = np.zeros_like(input_data, dtype=int)
    for s in sources:
        d = skimage.draw.disk(center=np.round(s) - .5, radius=radius, shape=a.shape)
        # for x, y in zip(townhall_disk[0] + s[0] - 20, townhall_disk[1] + s[1] - 20):
        #     if 0 <= x < a.shape[0] and 0 <= y < a.shape[1]:
        #         a[x, y] += 1
        a[d] = a[d] + 1
    return a

def constraints(x):
    coords = x.reshape((-1, 2))
    sources = vector_to_sources(x)
    for x, y in sources:
        r = skimage.draw.rectangle(
            start=np.array((x, y)) - source_size / 2,
            extent=(source_size, source_size),
            shape=input_data.shape,
        )
        samples = distance_grid[r]
        if samples.size == 0:
            samples = distance_grid
        yield max(0, samples.max())
        i = round(x)
        j = round(y)
        i2 = max(0, min(input_data.shape[0] - 1, i))
        j2 = max(0, min(input_data.shape[1] - 1, j))
        yield abs(i - i2) + abs(j - j2)

    distances = pairwise_distances(np.round(coords))
    distances_tril = distances[np.tril_indices_from(distances, -1)]
    distance_constraints = np.maximum(0, 2 * radius -  distances_tril)
    distance_constraints = np.where(0 < distance_constraints, np.maximum(1, distance_constraints), distance_constraints)
    yield distance_constraints.sum()

def reward_fn(x):
    global iteration
    sources = vector_to_sources(x)
    coverage = sources_to_grid(sources)
    reward_coverage = (input_data * (coverage > 0)).sum()
    # return reward_coverage
    loss_constraints = sum(constraints(x))
    reward = reward_coverage - 1e2 * loss_constraints
    return reward

In [14]:
def vector_to_image(x):
    sources = vector_to_sources(x)
    output_shape = input_data.shape[0], input_data.shape[1], 3
    img = np.zeros(output_shape, dtype=np.uint8)
    coverage = sources_to_grid(sources)
    img[:, :, 0] += (255 * (coverage == 0) * (input_data == 1)).astype(np.uint8)
    img[:, :, 1] += (255 * (coverage == 0) * (input_data == 1)).astype(np.uint8)
    img[:, :, 0] += (255 * (coverage > 1) * (input_data == 1)).astype(np.uint8)
    img[:, :, 1] += (255 * (coverage == 1) * (input_data == 1)).astype(np.uint8)
    for x, y in sources:
        
        r = skimage.draw.rectangle(
            start=np.array((x, y)) - source_size / 2,
            extent=(source_size, source_size),
            shape=input_data.shape,
        )
        img[r[0], r[1], :] = 255
    return img

In [15]:

iteration = 0
images = []
image_hashes = set()
def cb(x, *args):
    global iteration
    iteration += 1
    img = vector_to_image(x)
    # image_hash = hash(img.data.tobytes())
    # if image_hash in image_hashes:
    #     return
    # image_hashes.add(image_hash)
    img = np.repeat(img, 2, 0)
    img = np.repeat(img, 2, 1)
    img = np.pad(img, ((100, 0), (0, 0), (0, 0)))
    image = Image.fromarray(img)
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("arial.ttf", 16)
    draw.text(
        (0, 0),
        f"Iteration : {iteration}",
        font=font,
    )

    
    sources = vector_to_sources(x)
    coverage = sources_to_grid(sources)
    coverage_tiles = ((coverage > 0) & input_data).sum()
    draw.text(
        (0, 20),
        f"Tiles covered : {coverage_tiles}",
        font=font,
    )
    constraint_loss = sum(constraints(x))
    draw.text(
        (0, 40),
        f"Constraint loss : {round(constraint_loss, 3)}",
        font=font,
    )

    images.append(image)
    for fname in f"iterations/{iteration}.png", "latest.png":
        output_path = os.path.join("..", "resources", "optimization", fname)
        image.save(output_path)

In [16]:
x_tolsimir = np.array([
    [34, 76],
    [13, 111],
    [75, 78],
    [53, 112],
    [31, 147],
    [71, 149],
    [89, 186],
    [111, 152],
    [94, 115],
    [130, 188],
    [170, 190],
    [152, 153],
    [193, 157],
    [233, 163],
    [136, 116],
    [177, 120],
    [218, 125],
    [116, 81],
    [98, 43],
    [121, 9],
    [142, 44],
    [182, 49],
    [158, 82],
    [200, 86],
])

px.imshow(vector_to_image(x_tolsimir))

In [24]:
iteration = 0
cb(x_tolsimir)

In [17]:
reward_fn(x_tolsimir)

np.float64(25818.0)

In [18]:
sum(constraints(x_tolsimir))

np.float64(0.0)

In [19]:
cma.CMAOptions()

{'AdaptSigma': 'True  # or False or any CMAAdaptSigmaBase class e.g. CMAAdaptSigmaTPA, CMAAdaptSigmaCSA',
 'CMA_active': 'True  # negative update, conducted after the original update',
 'CMA_active_injected': '0  #v weight multiplier for negative weights of injected solutions',
 'CMA_cmean': '1  # learning rate for the mean value',
 'CMA_const_trace': 'False  # normalize trace, 1, True, "arithm", "geom", "aeig", "geig" are valid',
 'CMA_diagonal': '0*100*N/popsize**0.5  # nb of iterations with diagonal covariance matrix, True for always',
 'CMA_diagonal_decoding': '0  # learning rate multiplier for additional diagonal update',
 'CMA_eigenmethod': 'np.linalg.eigh  # or cma.utilities.math.eig or pygsl.eigen.eigenvectors',
 'CMA_elitist': 'False  #v or "initial" or True, elitism likely impairs global search performance',
 'CMA_injections_threshold_keep_len': '1  #v keep length if Mahalanobis length is below the given relative threshold',
 'CMA_mirrors': 'popsize < 6  # values <0.5 are int

In [20]:
options = cma.CMAOptions()
options.set("bounds", [bounds_lower, bounds_upper])
options.set("seed", 42)
options.set("popsize", 500)
options.set("tolflatfitness", 32)
# options.set("tolfun", 0)
# options.set("tolfunhist", 0)
options.set("integer_variables", list(range(num_variables)))

{'AdaptSigma': 'True  # or False or any CMAAdaptSigmaBase class e.g. CMAAdaptSigmaTPA, CMAAdaptSigmaCSA',
 'CMA_active': 'True  # negative update, conducted after the original update',
 'CMA_active_injected': '0  #v weight multiplier for negative weights of injected solutions',
 'CMA_cmean': '1  # learning rate for the mean value',
 'CMA_const_trace': 'False  # normalize trace, 1, True, "arithm", "geom", "aeig", "geig" are valid',
 'CMA_diagonal': '0*100*N/popsize**0.5  # nb of iterations with diagonal covariance matrix, True for always',
 'CMA_diagonal_decoding': '0  # learning rate multiplier for additional diagonal update',
 'CMA_eigenmethod': 'np.linalg.eigh  # or cma.utilities.math.eig or pygsl.eigen.eigenvectors',
 'CMA_elitist': 'False  #v or "initial" or True, elitism likely impairs global search performance',
 'CMA_injections_threshold_keep_len': '1  #v keep length if Mahalanobis length is below the given relative threshold',
 'CMA_mirrors': 'popsize < 6  # values <0.5 are int

In [21]:

# res = differential_evolution(
#     lambda x: -reward_fn(x),
#     bounds=bounds,
#     callback=cb,
#     constraints=scipy.optimize.NonlinearConstraint(lambda x: sum(constraints(x)), -np.inf, 1e-2),
#     disp=True,
#     integrality=True,
#     maxiter=10_000,
#     # tol=1e-8,
#     # strategy="randtobest1bin",
#     # popsize=32,
#     # workers=2,
#     # atol=1.0,
#     # tol=1e-5,
#     # recombination=.9,
# )

In [22]:



# res = scipy.optimize.dual_annealing(
#     loss_fn,
#     bounds=bounds,
#     callback=cb,
#     maxiter=10_000,
# )

x0=[(li + ui) / 2 for li, ui in bounds]
sigma0=math.sqrt((input_data.shape[0] * input_data.shape[1]) / 12)

# x0 = np.array([ 48.86787419, 145.05459966, 150.06288545,  47.92156405,
#        138.99561885,  87.28520437,  71.10871424,  68.33506996,
#         21.76786652, 115.05052127, 148.73939368, 194.0049809 ,
#        190.24731688,  38.854061  , 110.31125759,  59.09129294,
#        196.97066102, 144.4597769 , 178.05749153,  77.22961749,
#        128.83280234, 126.21920866, 224.49979488, 173.93801628,
#        157.65373721, 155.09534156, 235.95264006, 134.9194016 ,
#        118.57426577, 164.97182244,  61.08120205, 107.14704587,
#         31.14791093,  76.45051155,  88.7839089 , 137.4468036 ,
#        100.15640686,  98.22663478, 206.91228001, 105.07770597,
#          8.54136674, 152.82120424, 121.48876576,  19.50708521,
#        167.73447415, 116.44081604,  81.68628624, 180.17291539])
# sigma=1.0

es = cma.CMAEvolutionStrategy(x0, sigma0, options)

objective_fn = lambda x: -reward_fn(x)

while not es.stop():
    solutions = es.ask()
    es.tell(solutions, list(map(objective_fn, solutions)))
    es.disp()
    cb(es.best.x)

# res = cma.fmin2(
#     objective_function=lambda x: -reward_fn(x),
#     x0=x0,
#     sigma0=sigma0,
#     # constraints=lambda x: list(constraints(x)),
#     # find_feasible_first=True,
#     # find_feasible_final=True,
#     restarts=5,
#     restart_from_best=True,
#     callback=lambda es: cb(es.result.xbest),
#     options=options,
# )

# res

(250_w,500)-aCMA-ES (mu_w=128.6,w_1=2%) in dimension 48 (seed=42, Thu Oct  2 18:08:07 2025)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1    500 6.990216873337107e+04 1.0e+00 8.47e+01  8e+01  9e+01 0:01.3
    2   1000 6.526285883148396e+04 1.1e+00 1.20e+02  1e+02  1e+02 0:02.6


KeyboardInterrupt: 

In [ ]:

# res = scipy.optimize.dual_annealing(
#     lambda x: -reward_fn(x),
#     bounds=bounds,
#     callback=cb,
#     maxiter=10_000,
#     x0=es.best.x,
# )
# res.x

In [ ]:

animation_path = os.path.join("..", "resources", "optimization", "animation.gif")
images[0].save(animation_path, save_all=True, append_images=images[1:], duration=len(images)/100, loop=0)